In [70]:
import os
import sys
from collections import defaultdict
from functools import reduce, partial

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO

In [142]:
obs_muts = ["195C", "263G", "315.1C", "499A", "16179T", "16356C", "16519C"]
dloop_positions = list(range(1, 577)) + list(range(16024, 16570))

In [74]:
refseq = str(next(SeqIO.parse("../../mtdnaMutSpecOfCancers/data/external/NC_012920.1.gb", format="gb")).seq)
len(refseq)

16569

In [119]:
rdf = pd.concat([pd.read_csv(f"../data/dloop/fg/hapl{i}.tsv", sep="\t") for i in [1,2,3,4,5,6,7]])
rdf.shape

(55878, 10)

In [120]:
rdf.iloc[0].to_dict()

{'SampleID': 'AP023485.1',
 'Haplogroup': 'D4b1b1a',
 'Rank': 1,
 'Quality': 0.99,
 'Range': '1-16569',
 'Not_Found_Polys': nan,
 'Found_Polys': '73G 263G 431T 489C 750G 1438G 2706G 3010A 4769G 4859C 4883T 5178A 6410T 6689T 7028T 8020A 8414T 8701G 8860G 9540C 10084C 10181T 10398G 10400T 10873C 11719A 12705T 14668T 14766T 14783C 15043A 15301A 15326G 15440C 15951G 16223T 16287T 16319A 16362C 16399G',
 'Remaining_Polys': '309.1C (hotspot) 315.1C (hotspot) 523d (hotspot) 524d (hotspot) 3107d (hotspot) 4025T (localPrivateMutation)',
 'AAC_In_Remainings': '4025T [T240M| Codon 2 | ND1 ]',
 'Input_Sample': '73G 263G 309.1C 315.1C 431T 489C 523d 524d 750G 1438G 2706G 3010A 3107d 4025T 4769G 4859C 4883T 5178A 6410T 6689T 7028T 8020A 8414T 8701G 8860G 9540C 10084C 10181T 10398G 10400T 10873C 11719A 12705T 14668T 14766T 14783C 15043A 15301A 15326G 15440C 15951G 16223T 16287T 16319A 16362C 16399G'}

In [121]:
rdf["Input_Sample"] = rdf.Input_Sample.str.split()
df = rdf[['SampleID', 'Haplogroup', 'Quality', 'Range', 'Input_Sample']].explode("Input_Sample").dropna()
df["Pos"] = df.Input_Sample.str.extract("(\d+).+").astype(np.int32)

In [122]:
df

,SampleID,Haplogroup,Quality,Range,Input_Sample,Pos
0,AP023485.1,D4b1b1a,0.99,1-16569,73G,73
0,AP023485.1,D4b1b1a,0.99,1-16569,263G,263
0,AP023485.1,D4b1b1a,0.99,1-16569,309.1C,309
0,AP023485.1,D4b1b1a,0.99,1-16569,315.1C,315
0,AP023485.1,D4b1b1a,0.99,1-16569,431T,431
...,...,...,...,...,...,...
2535,KX557487.1,H35,1.00,1-16569,4769G,4769
2535,KX557487.1,H35,1.00,1-16569,7585.1T,7585
2535,KX557487.1,H35,1.00,1-16569,8860G,8860
2535,KX557487.1,H35,1.00,1-16569,15326G,15326


In [123]:
df[df.Pos == 524].Input_Sample.value_counts()

524d         13475
524.1AC       1446
524.1ACAC      458
524G            48
524.1NN         19
524.1CA         11
524T             8
524.1A           8
524A             5
524.1CACA        2
524.1ACCA        1
524.1ACAS        1
Name: Input_Sample, dtype: int64

In [124]:
refseq[524 - 1 +1]

'C'

In [146]:
dloop_muts = df[df.Pos.isin(dloop_positions)].groupby("SampleID").apply(lambda x: list(x.Input_Sample.values))

In [147]:
dloop_muts.apply(lambda x: x == obs_muts).sum()

0

In [127]:
df[df.Input_Sample.isin(obs_muts)].groupby("SampleID").Haplogroup.count().value_counts().sort_index()#.plot.bar()

1     6000
2    21631
3    20987
4     5486
5      354
6      287
7       51
Name: Haplogroup, dtype: int64

In [143]:
d = df[df.Input_Sample.isin(obs_muts)].groupby("SampleID").Haplogroup.count().sort_values()

In [150]:
sim_genomes = df[(df.Pos.isin(dloop_positions)) & (df.SampleID.isin(d[d == 7].index.values))].groupby("SampleID").apply(lambda x: list(x.Input_Sample.values))#.Input_Sample.value_counts()

In [114]:
len(obs_muts)

8

In [118]:
1500142 -800026

700116